# Exercise 29 (1) - Data-Driven Solver: generate a data set
### Task
Generate a data set with the code below

### Learning goals
- Familiarize yourself with a data generation pipeline in simulation

In [ ]:
import torch
import pandas as pd
import numpy as np
import time

In [ ]:
import FiniteDifference

## Pre-processing

**loading settings of measurements**

In [ ]:
settings = pd.read_csv("dataset1DFWI/settings.csv")

numberOfSamples = settings.numberOfSamples[0]

Lx = settings.Lx[0]
Nx = settings.Nx[0]
dx = Lx / Nx
dt = settings.dt[0]
N = settings.N[0]
c0 = settings.c0[0]

**generate grid**

In [ ]:
x = np.linspace(0 - dx, Lx + dx, Nx + 3)  # with ghost cells
t = np.linspace(0, (N - 1) * dt, N)
x_, t_ = np.meshgrid(x, t, indexing='ij')

**define source**

In [ ]:
def diracx(x, i):
    x = x * 0
    x[i] = 1
    return x


def generateSineBurst(frequency, cycles):
    omega = frequency * 2 * np.pi
    return lambda t: 1e14 * ((t <= cycles / frequency) & (t > 0)) * np.sin(omega * t) * (
        np.sin(omega * t / 2 / cycles)) ** 2


frequency = settings.frequency[0]
cycles = settings.cycles[0]

**define experiment in terms of source and sensor placement**

In [ ]:
sourcePositions = (1, -2)
sensorPositions = (1, -2)

## Data generation

In [ ]:
for j in range(numberOfSamples):

    minimumSizeOfDefect = 0.01  # in percent
    defect = np.random.rand(3)  # uniform distribution in range 0, 1

    defect[0] *= Lx * (1 - minimumSizeOfDefect)
    defect[1] = defect[0] + np.max([defect[1] * (Lx - defect[0]), Lx * minimumSizeOfDefect])
    defect[2] = np.max([defect[2] * c0, 100])

    index = (x > defect[0]) & (x < defect[1])
    c = x * 0 + c0
    c[index] = defect[2]

    U = np.zeros((len(sourcePositions), len(sensorPositions), N + 1))

    for i in range(len(sourcePositions)):

        source = generateSineBurst(frequency, cycles)
        f_source = lambda x, t, i: diracx(x, i) * source(t)
        f = f_source(x_, t_, sourcePositions[i])[1:-1, :]

        u0 = x * 0
        u1 = x * 0

        start = time.perf_counter()
        u = FiniteDifference.finiteDifference1D(u0.copy(), u1.copy(), f, c, dx, Nx, dt, N)
        U[i] = u[sensorPositions, :]
        end = time.perf_counter()
        if (j % 100 == 0) & (i == 0):
            print("Elapased time of sample {:d}: {:2f} ms".format(j, (end - start) * 1000))

        if j == 0:
            pd.DataFrame(f).to_hdf("dataset1DFWI/source" + str(i) + ".h5", key='F', index=False, mode='w', complevel=1)

    # export data
    torch.save(torch.as_tensor(U).to(torch.float32), "dataset1DFWI/measurement" + str(j) + ".pt")
    torch.save(torch.as_tensor(c).to(torch.float32), "dataset1DFWI/material" + str(j) + ".pt")
    torch.save(torch.as_tensor(defect).to(torch.float32), "dataset1DFWI/materialCoefficients" + str(j) + ".pt")